In [1]:
import json
import re
from gensim.models import Word2Vec
import csv


In [2]:
f = open('digital_trajectoriesraject_public_keyword.json')
data = json.load(f)

f.close()

In [3]:
f = open('documents1.json')
documents = json.load(f)

f.close()

In [4]:
words_only = []

with open('words_only.csv', newline='\n') as csvfile:
    reader = csv.reader(csvfile, delimiter="'", quotechar='|')
    words_only = [row for row in reader]
    
words = []

for i in words_only:
    arr = []
    for j in i:
        if j != '' and j != ', ' and j != '[]' and j != '[' and j != ']':
            arr.append(j)
    #arr = [j  for j in i]
    words.append(arr)
    
words_no_repeat = []

for i in words:
    words_no_repeat.append(list(set(i)))
    
all_in_one = []

for i in words_no_repeat:
    for j in i:
        all_in_one.append(j)
        
all_in_one = list(set(all_in_one))


In [6]:
arr = []
for i in all_in_one:
    i = re.sub('[^а-яёa-z -+/]', ' ', i.lower())
    i = re.sub(r'\s+', ' ', i)
    arr.append(i)
    
all_in_one = [] + arr

all_in_one[:10]

['нейросетевое',
 'gephi',
 'rstudio',
 'рачинского',
 'estimate activity durations',
 'stl',
 'autosar adaptive',
 'индустрии',
 'matlab',
 'microsoft solutions framework']

In [35]:
'ros' in all_in_one

True

https://python-school.ru/blog/word2vec-with-examples-in-gensim/

In [7]:
text = ""
for doc in documents: text += doc['text']

In [8]:
#проверить на точки вида node.js
sentences = text.lower().split('.')

In [9]:
sentences[:5]

['коммерциализация результатов научных исследований и разработок представляет собой процесс вовлечения их в экономический (коммерческий) оборот в различных сегментах национального и глобального рынков',
 ' актуальность данной дисциплины обусловлена становлением на путь модернизации экономики страны',
 ' в сложившейся ситуации резко возрастает необходимость оценки и обоснования экономической эффективности проектов, ориентированных на выпуск высокотехнологичной продукции и продвижение новых технологий',
 ' рассматривается действующее законодательство российской федерации в области регулирования деятельности предприятий втоэ и защиты их интеллектуальной собственности',
 '']

In [10]:
words = []
for i in sentences:
    i = re.sub('[^а-яёa-z -+/]', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    words.append(i.split(' '))

In [13]:
w2v_model = Word2Vec(
    min_count=10,
    window=2,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [14]:
w2v_model.build_vocab(words)

In [15]:
w2v_model.train(words, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(71157507, 131504460)

In [16]:
w2v_model.init_sims(replace=True)


/tmp/ipykernel_87315/2897808894.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [17]:
w2v_model.wv.most_similar("sql", topn=20)

[('инъекций', 0.6878746151924133),
 ('инъекция', 0.6429065465927124),
 ('генерировала', 0.6088489294052124),
 ('слепая', 0.6030614376068115),
 ('реляционными', 0.5580300092697144),
 ('инъекции', 0.5542211532592773),
 ('limbo', 0.5485631823539734),
 ('диалекты', 0.5404766201972961),
 ('бд', 0.5380511283874512),
 ('foundation', 0.5346158146858215),
 ('запросов', 0.5260201692581177),
 ('sparql', 0.5199368596076965),
 ('реляционных', 0.5184134244918823),
 ('server', 0.516451358795166),
 ('предикатов', 0.5157637596130371),
 ('nosql', 0.5154807567596436),
 ('mvc', 0.5145859122276306),
 ('предложенные', 0.5126041769981384),
 ('team', 0.5109604597091675),
 ('реляционная', 0.509156346321106)]

In [18]:
w2v_model.wv.n_similarity(["баз", "данных"], ["sql"])

0.4745509

In [19]:
synonyms = {}
for d in data:
    synonyms[d['value']] = []

In [20]:
for i in range(200,500):
    for j in range(i+1, 500):
        v1 = data[i]['value']
        v2 = data[j]['value']
        sim = w2v_model.wv.n_similarity(v1.lower().split(' '), v2.lower().split(' '))
        if sim > 0.6 and sim < 1:
            synonyms[v1].append(v2)
            synonyms[v2].append(v1)

In [21]:
for i in data[200:210]:
    print(i['value'])
    print(synonyms[i['value']], '\n')

методики обработки информации
['методики представления данных', 'методики преобразования моделей', 'методики доступа', 'методики использования', 'освоение методов представления знаний', 'средства манипулирования наборами знаний', 'инструменты проектирования графов знаний', 'методики преобразования баз данных', 'описание наборов данных rdf', 'средства построения систем ии', 'выбор методов тестирования', 'развитие задач тестирования', 'разработки архитектуры систем', 'методов наполнения данными графов знаний', 'области представления знаний', 'разработки информационных', 'технологий решения', 'изучение инструментов сбора', 'сбор данных пациента', 'обработка данных диагностики', 'инструменты получения данных', 'основе данных диагностики пациента', 'прототипированию данных', 'данных анатомической', 'формирование навыков использования ряда', 'возможность интерпретации знаний', 'методы обеспечения безопасности передачи трафика', 'построения систем сбора', 'основы передачи данных', 'механизмы 

In [22]:
#for i in data[200:350]:
    #print(i['value'])
    #print(len(synonyms[i['value']]), "\t", len(synonyms[i['value']])/300, '\n')

In [23]:
synonyms['язык sql']

['язык представления онтологий owl']

In [24]:
sim = w2v_model.wv.n_similarity(['язык', 'sql'], "системы баз данных".lower().split(' '))
sim

0.3778696

In [25]:
wikin_synonyms = {}
for i in all_in_one:
    wikin_synonyms[i] = []

In [26]:
l = len(all_in_one)

for i in range(l):
    for j in range(i+1, l):
        v1 = all_in_one[i]
        v2 = all_in_one[j]
        sim = w2v_model.wv.n_similarity(v1.lower().split(' '), v2.lower().split(' '))
        if sim > 0.5 and sim < 1:
            wikin_synonyms[v1].append(v2)
            wikin_synonyms[v2].append(v1)

In [27]:
weights = {}
l_all_in_one = len(all_in_one)

for i in wikin_synonyms:
    print(i)
    print(wikin_synonyms[i])
    print()
    weights[i] = len(i)/l_all_in_one

нейросетевое
[]

gephi
[]

rstudio
['anylogic', 'business intelligence', 'anylogic', 'business intelligence']

рачинского
['microsoft solutions framework', 'tasks', 'product owner', 'backlog', 'андерсона дарлинга', 'карацубы', 'team', 'user story', 'scrum master', 'михайлова', 'м ю xa шестопалов']

estimate activity durations
['autosar adaptive', 'microsoft solutions framework', 'service oriented architecture', 'tasks', 'product owner', 'team foundation server', 'backlog', 'business intelligence', 'fuzzy ', 'federated ai technology enabler', 'rational unified process', 'network science', 'message passing interface', 'team', 'opensplice', 'autosap adaptive platform', 'вороного', 'constraint programming', 'user story', 'tensorflow federated', 'ansi pmi pmbok th edition', 'scrum master', 'publisher xadsubscriber', 'robot operating system', 'simultaneous localization and mapping', 'sprint burndown chart', 'autosar adaptive platform', 'rti connext dds', 'simultaneous localization and mappin

In [28]:
wikin_synonyms2 = {}
for i in all_in_one:
    wikin_synonyms2[i] = []
    

for i in range(l):
    for j in range(i+1, l):
        v1 = all_in_one[i]
        v2 = all_in_one[j]
        sim = w2v_model.wv.n_similarity(v1.lower().split(' '), v2.lower().split(' '))
        s = 0.5
        if weights[v1] > 0.5 or weights[v2] > 0.5:
            s = 0.75
        elif weights[v1] > 0.25 or weights[v2] < 0.25:
            s = 0.65
        elif weights[v1] > 0.1 or weights[v2] < 0.1:
            s = 0.6
        elif weights[v1] < 0.03 or weights[v2] < 0.03:
            s = 0.45
        elif weights[v1] < 0.01 or weights[v2] < 0.01:
            s = 0.4
        if sim > s and sim < 1:
            wikin_synonyms2[v1].append(v2)
            wikin_synonyms2[v2].append(v1)

In [29]:
wikin_synonyms2

{'нейросетевое': [],
 'gephi': [],
 'rstudio': ['anylogic', 'anylogic'],
 'рачинского': ['backlog'],
 'estimate activity durations': ['tasks',
  'product owner',
  'business intelligence',
  'federated ai technology enabler',
  'network science',
  'simultaneous localization and mapping',
  'simultaneous localization and mapping'],
 'stl': [],
 'autosar adaptive': ['autosar adap tive',
  'autosap adaptive platform',
  'autosar adaptive platform'],
 'индустрии': [],
 'matlab': [],
 'microsoft solutions framework': ['team foundation server',
  'rational unified process',
  'team'],
 'государственная': [],
 'service oriented architecture': ['tasks',
  'team foundation server',
  'message passing interface',
  'team',
  'autosap adaptive platform'],
 'tasks': ['estimate activity durations',
  'service oriented architecture',
  'product owner',
  'business intelligence',
  'federated ai technology enabler',
  'network science',
  'team',
  'estimate activity resources',
  'tensorflow federa

In [30]:
#for i in range(l):
#    w = all_in_one[i]
#    print(w)
#    print(len(wikin_synonyms[w]), "vs", len(wikin_synonyms2[w]))

In [33]:
with open('wikiner_synonyms05.json', 'w') as outfile:
    outfile.write(json.dumps(wikin_synonyms, ensure_ascii=False, indent=4))


In [34]:
with open('wikiner_synonyms04_075.json', 'w') as outfile:
    outfile.write(json.dumps(wikin_synonyms2, ensure_ascii=False, indent=4))